Så prøver vi en gang til at få det her shizzle til at virke. Først alle import

In [2]:
import os
import sys

In [3]:
os.chdir('/home/au643300/DataHandling/')

Sætter Datahandling in i min syspath

In [4]:
sys.path.append('/home/au643300/DataHandling/')

In [5]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed
import glob
from src.data.tonetCDF import to_netcdf
import xarray as xr

In [6]:
def save_func(xarray,filename):
    import xarray as xr
    save_path = "/home/au643300/NOBACKUP/interim/"
    xarray.to_netcdf(save_path + filename+ 'nc', engine='netcdf4')


starter cluster op

In [7]:
cluster=SLURMCluster(cores=4,
                     processes=1,
                     memory="22GiB",
                     queue='q64',
                     walltime='0-00:30:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     extra=['--resources mem=15']
                    )
                    

In [8]:
client=Client(cluster)

In [9]:
cluster.adapt(minimum_jobs=1,maximum_jobs=5)

In [10]:
client

Client Scheduler: tcp://10.100.8.14:36722 Dashboard: http://10.100.8.14:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


Deleting the files smaller than 500 mb, as they are defect

In [11]:
raw_path="/home/au643300/DataHandling/data/raw/"
files_raw=glob.glob(raw_path+'*.u')

interim_path="/home/au643300/NOBACKUP/interim/"
files_interim=glob.glob(interim_path+'*.nc')
for file in files_interim:
    size=os.path.getsize(file)
    if size < 500000000:
        os.remove(file)



Now finding the missing files


In [12]:
file_only_interim=[]
file_only_raw=[]
for file in files_interim:
    a=os.path.basename(file)
    a=a[:-3]
    file_only_interim.append(a)
for file in files_raw:
    b=os.path.basename(file)
    b=b[:-2]
    file_only_raw.append(b)

del a,b




diff=list(set(file_only_interim)^set(file_only_raw))

In [13]:
sym_path=[]
for name in diff:
    sym_path.append("/home/au643300/DataHandling/data/raw/"+name+'.u')

file_path=[]
for path in sym_path:
    file_path.append(os.readlink(path))





In [ ]:
ds=client.map(to_netcdf,file_path,resources={'mem': 5},retries=5)

for future,result in as_completed(ds, with_results=True):
    ds,file = result
    ds.to_netcdf(interim_path + file+ 'nc', engine='netcdf4')



